In [1]:
from os import listdir
from os.path import isfile, join
from openpyxl import load_workbook
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime 
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from sklearn.cluster import KMeans
import numpy as np
import datetime
import calendar
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


## Getting Sales Data For Last Date 

In [3]:
root_path = r"C:\Users\K-M\Desktop\TL\Final Shape of Files"
os.chdir(root_path)
Sales_full = 'Sales_full'
Sales_mini = 'Sales_mini'
df_sf = pd.read_csv(Sales_full)
df_sm = pd.read_csv(Sales_mini)
df_sf.fillna(0, inplace = True)
df_sm.fillna(0, inplace = True)
df_sf['Date'] = pd.to_datetime(df_sf['Date'], format =  "%d/%m/%Y")
df_sm['Date'] = pd.to_datetime(df_sm['Date'], format =  "%d/%m/%Y")

In [4]:
root_path = r"C:\Users\K-M\Desktop\discontinue"
os.chdir(root_path)
df_tl = pd.read_excel('List of All Products - NEW.xlsx')

In [5]:
workbook = load_workbook('List of All Products - NEW.xlsx')

# Get all sheet names
sheet_names = workbook.sheetnames
df_tl = pd.DataFrame()
for i in sheet_names[:4]:
    df_t = pd.read_excel('List of All Products - NEW.xlsx', sheet_name = i)
    df_t['Country'] = i
    df_tl = pd.concat([df_tl, df_t])

### Getting the start date of each product

In [7]:
start_date = df_sm[['Date','ASIN','SKU', 'Marketplace']].sort_values(by = 'Date').drop_duplicates(subset =['ASIN','SKU', 'Marketplace'] ,keep = 'first')

## Getting Revenues and profit per year

In [9]:
df_sm = df_sm[df_sm['Date'].between('2023-01-01', '2024-12-31')]
df_sf = df_sf[df_sf['Date'].between('2023-01-01', '2024-12-31')]
df_sf['Gross Revenue'] = df_sf['SalesOrganic'] +  df_sf['SalesSponsoredProducts'] + df_sf['SalesSponsoredDisplay']
df_sf['Units Sold'] = df_sf['UnitsOrganic'] + df_sf['UnitsPPC']
df_sf['FBA storage fee'] = df_sf['FBALongTermStorageFee'] + df_sf['FBAStorageFee'] +  df_sf['FBAPerUnitFulfillmentFee']


In [10]:
df_sf.sort_values('Date')
df_sf['year'] = df_sf['Date'].dt.year
df_sm['year'] = df_sm['Date'].dt.year

In [11]:
df_r = pd.pivot_table(df_sm,index=['SKU', 'ASIN', 'year', 'Marketplace'], values=(['RefundCost','NetProfit']), aggfunc = 'sum')
df_r2 = pd.pivot_table(df_sf,index=['SKU', 'ASIN', 'year', 'Marketplace'], values=(['Gross Revenue','Units Sold']), aggfunc = 'sum')

In [12]:
df_r = df_r.reset_index()
df_r2 = df_r2.reset_index()

In [13]:
df_r2023 = df_r[df_r['year'] == 2023]
df_r22023 = df_r2[df_r2['year'] == 2023]
df_r2023.rename(columns = {'NetProfit': 'NetProfit 2023', 'RefundCost': 'RefundCost 2023'}, inplace = True)
df_r22023.rename(columns = {'Gross Revenue': 'Gross Revenue 2023', 'Units Sold': 'Units Sold 2023'}, inplace = True)


df_r2024 = df_r[df_r['year'] == 2024]
df_r22024 = df_r2[df_r2['year'] == 2024]
df_r2024.rename(columns = {'NetProfit': 'NetProfit 2024', 'RefundCost': 'RefundCost 2024'}, inplace = True)
df_r22024.rename(columns = {'Gross Revenue': 'Gross Revenue 2024', 'Units Sold': 'Units Sold 2024'}, inplace = True)


C:\Users\K-M\AppData\Local\Temp\ipykernel_7104\4051028930.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_r2023.rename(columns = {'NetProfit': 'NetProfit 2023', 'RefundCost': 'RefundCost 2023'}, inplace = True)
C:\Users\K-M\AppData\Local\Temp\ipykernel_7104\4051028930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_r22023.rename(columns = {'Gross Revenue': 'Gross Revenue 2023', 'Units Sold': 'Units Sold 2023'}, inplace = True)
C:\Users\K-M\AppData\Local\Temp\ipykernel_7104\4051028930.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

In [14]:
df = df_r2024.merge(df_r22024.drop(columns = 'year'), on = ['Marketplace', 'ASIN', 'SKU'], how = 'left')
df  =df.merge(df_r2023.drop(columns = 'year'), on = ['Marketplace', 'ASIN', 'SKU'], how = 'left')
df = df.merge(df_r22023.drop(columns = 'year'), on = ['Marketplace', 'ASIN', 'SKU'], how = 'left')
df = df.merge(start_date, on = ['Marketplace', 'ASIN', 'SKU'], how = 'left')

# Aggregating Based on Product Name


In [16]:
df_sku_level = df.copy()

In [17]:
df_A = pd.pivot_table(df,index=['Marketplace','ASIN'], values=(['NetProfit 2024',
       'RefundCost 2024', 'Gross Revenue 2024', 'Units Sold 2024',
       'NetProfit 2023', 'RefundCost 2023', 'Gross Revenue 2023',
       'Units Sold 2023']), aggfunc = 'sum').reset_index()

df_A = df_A.merge(start_date.drop(columns = 'SKU').drop_duplicates(subset = ['Marketplace', 'ASIN'], keep = 'first'), on = ['Marketplace', 'ASIN'], how = 'left')

In [18]:
df_A['Net Margin 2023'] = df_A['NetProfit 2023'] / df_A['Gross Revenue 2023']
df_A['Net Margin 2024'] = df_A['NetProfit 2024'] / df_A['Gross Revenue 2024']
df_A.replace([np.inf, -np.inf], 0, inplace=True)


In [19]:
df_A['Country'] = df_A['Marketplace']
df_A['Country'].replace( {'Amazon.ca':'CA', 'Amazon.co.uk': 'UK', 'Amazon.com': 'USA', 'Amazon.com.be': 'EU',
       'Amazon.com.mx': 'USA', 'Amazon.com.tr': 'EU', 'Amazon.de': 'EU', 'Amazon.es': 'EU',
       'Amazon.fr': 'EU', 'Amazon.it': 'EU', 'Amazon.nl': 'EU', 'Amazon.pl': 'EU', 'Amazon.se': 'EU'}, inplace = True)


C:\Users\K-M\AppData\Local\Temp\ipykernel_7104\3794279897.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_A['Country'].replace( {'Amazon.ca':'CA', 'Amazon.co.uk': 'UK', 'Amazon.com': 'USA', 'Amazon.com.be': 'EU',


In [20]:
df_A.reset_index(inplace = True, drop = True)

In [21]:
for i in range(0, df_A.shape[0]):
    try:
        df_t = df_tl[(df_tl['ASIN'] == df_A.loc[i, 'ASIN']) & (df_tl['Country'] == df_A.loc[i, 'Country'])]
        df_A.loc[i, 'Product Name'] = df_t['Products (Other name)'].unique()
    except:
        pass

df_A = df_A[df_A['Product Name'].isna() == False]


In [22]:
df_A.reset_index(inplace  = True, drop = True)
df_A['Product Name'] = df_A['Product Name'].astype(str).str.strip()

In [23]:
df_A2 = pd.pivot_table(df_A,index=['Product Name', 'Marketplace'], values=(['NetProfit 2024',
       'RefundCost 2024', 'Gross Revenue 2024', 'Units Sold 2024',
       'NetProfit 2023', 'RefundCost 2023', 'Gross Revenue 2023',
       'Units Sold 2023']), aggfunc = 'sum').reset_index()
df_All = pd.pivot_table(df_A,index=['Product Name'], values=(['NetProfit 2024',
       'RefundCost 2024', 'Gross Revenue 2024', 'Units Sold 2024',
       'NetProfit 2023', 'RefundCost 2023', 'Gross Revenue 2023',
       'Units Sold 2023']), aggfunc = 'sum').reset_index()

In [24]:
start_date_new = df_A[['Product Name',
                       'Date']].sort_values(by = 'Date').reset_index(drop = True).drop_duplicates(subset = 'Product Name',keep = 'first')
df_All = df_All.merge(start_date_new, how = 'left', on = ['Product Name'])

In [25]:
start_date_new = df_A[['Product Name', 'Marketplace',
                       'Date']].sort_values(by = 'Date').reset_index(drop = True).drop_duplicates(subset = ['Product Name',
                                                                                                            'Marketplace'],keep = 'first')
df_A2 = df_A2.merge(start_date_new, how = 'left', on = ['Marketplace', 'Product Name'])

In [26]:
df_All['Net Margin 2023'] = df_All['NetProfit 2023'] / df_All['Gross Revenue 2023']
df_All['Net Margin 2024'] = df_All['NetProfit 2024'] / df_All['Gross Revenue 2024']
df_A2['Net Margin 2023'] = df_A2['NetProfit 2023'] / df_A2['Gross Revenue 2023']
df_A2['Net Margin 2024'] = df_A2['NetProfit 2024'] / df_A2['Gross Revenue 2024']
df_All.fillna(0, inplace = True)
df_A2.fillna(0, inplace = True)

In [27]:
df_A2['Sales more than 100 units'] = df_A2['Units Sold 2024'] > 100
df_A2['Net Profit above $200'] = df_A2['NetProfit 2024'] > 200
df_A2['Net Margin above 10%'] = df_A2['Net Margin 2024'] > 0.1

df_All['Sales more than 100 units'] = df_All['Units Sold 2024'] > 100
df_All['Net Profit above $200'] = df_All['NetProfit 2024'] > 200
df_All['Net Margin above 10%'] = df_All['Net Margin 2024'] > 0.1

In [28]:
product_new = df_A2[df_A2['Date'] < '2024-01-01']
product_new['Yearly Gross Revenue growth'] = product_new['Gross Revenue 2024'] > product_new['Gross Revenue 2023']
product_new['Yearly NetProfit growth'] = product_new['NetProfit 2024'] > product_new['NetProfit 2023']
product_new['Score'] = product_new[['Sales more than 100 units', 'Net Profit above $200',
       'Net Margin above 10%', 'Yearly Gross Revenue growth',
       'Yearly NetProfit growth']].sum(axis = 1)
product_new =product_new [[ 'Product Name', 'Date', 'Marketplace',
                       'Gross Revenue 2024', 'Gross Revenue 2023',
                       'NetProfit 2024', 'NetProfit 2023', 
                       'RefundCost 2024', 'RefundCost 2023', 
                       'Units Sold 2024', 'Units Sold 2023',
                       'Net Margin 2024','Net Margin 2023', 
                       'Sales more than 100 units', 'Net Profit above $200',
                       'Net Margin above 10%', 'Yearly Gross Revenue growth',
                       'Yearly NetProfit growth', 'Score']]
#ind = df_A2[df_A2['Date'] > '2024-01-01'].index
#df_A2.drop(ind, inplace = True)
df_A2.reset_index(inplace = True, drop = True)
product_new.reset_index(inplace = True, drop = True)
product_new.drop(columns = ['NetProfit 2023', 'Gross Revenue 2023', 'RefundCost 2023', 'Units Sold 2023','Net Margin 2023' ], inplace = True)
df_A2['Score'] = df_A2[['Sales more than 100 units', 'Net Profit above $200',
       'Net Margin above 10%']].sum(axis = 1)

C:\Users\K-M\AppData\Local\Temp\ipykernel_7104\2866378349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_new['Yearly Gross Revenue growth'] = product_new['Gross Revenue 2024'] > product_new['Gross Revenue 2023']
C:\Users\K-M\AppData\Local\Temp\ipykernel_7104\2866378349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_new['Yearly NetProfit growth'] = product_new['NetProfit 2024'] > product_new['NetProfit 2023']
C:\Users\K-M\AppData\Local\Temp\ipykernel_7104\2866378349.py:4: SettingWit

In [29]:
for i in range(0, product_new.shape[0]):
    if product_new.loc[i, 'Score'] == 0 or product_new.loc[i, 'Score'] == 1:
        product_new.loc[i, 'Class'] = 'D' 
    if product_new.loc[i, 'Score'] == 2 or product_new.loc[i, 'Score'] == 3:
        product_new.loc[i, 'Class'] = 'C' 
    if product_new.loc[i, 'Score'] == 4:
        product_new.loc[i, 'Class'] = 'B' 
    if product_new.loc[i, 'Score'] == 5:
        product_new.loc[i, 'Class'] = 'A' 


for i in range(0, df_A2.shape[0]):
    if df_A2.loc[i, 'Score'] == 0:
        df_A2.loc[i, 'Class'] = 'D' 
    if df_A2.loc[i, 'Score'] == 1:
        df_A2.loc[i, 'Class'] = 'C' 
    if df_A2.loc[i, 'Score'] == 2:
        df_A2.loc[i, 'Class'] = 'B' 
    if df_A2.loc[i, 'Score'] == 3:
        df_A2.loc[i, 'Class'] = 'A' 

In [30]:
df_A2 = df_A2[['Product Name', 'Marketplace', 'Date', 'Gross Revenue 2024',
       'NetProfit 2024', 'Units Sold 2024',
       'Net Margin 2024', 'Sales more than 100 units', 'Net Profit above $200',
       'Net Margin above 10%', 'Score', 'Class']]

In [31]:
product_new.sort_values(by = ['Score',  'Class', 'Gross Revenue 2024'], ascending = False).to_excel('Discontinuation per marketplace Report.xlsx',
                                                                                                   sheet_name = 'Old Products')
with pd.ExcelWriter('Discontinuation per marketplace Report.xlsx', engine='openpyxl', mode='a') as writer:
    for i in df_A['Marketplace'].unique():
        df_t = df_A2[df_A2['Marketplace'] == i]
        df_t.sort_values(by = ['Score',  'Class', 'Gross Revenue 2024'], ascending = False).to_excel(writer, sheet_name= i,  index = False)

In [32]:
product_new = df_All[df_All['Date'] < '2024-01-01']
product_new['Yearly Gross Revenue growth'] = product_new['Gross Revenue 2024'] > product_new['Gross Revenue 2023']
product_new['Yearly NetProfit growth'] = product_new['NetProfit 2024'] > product_new['NetProfit 2023']
product_new['Score'] = product_new[['Sales more than 100 units', 'Net Profit above $200',
       'Net Margin above 10%', 'Yearly Gross Revenue growth',
       'Yearly NetProfit growth']].sum(axis = 1)
product_new =product_new [[ 'Product Name', 'Date',
                       'Gross Revenue 2024', 'Gross Revenue 2023',
                       'NetProfit 2024', 'NetProfit 2023', 
                       'RefundCost 2024', 'RefundCost 2023', 
                       'Units Sold 2024', 'Units Sold 2023',
                       'Net Margin 2024','Net Margin 2023', 
                       'Sales more than 100 units', 'Net Profit above $200',
                       'Net Margin above 10%', 'Yearly Gross Revenue growth',
                       'Yearly NetProfit growth', 'Score']]

df_All.reset_index(inplace = True, drop = True)
product_new.reset_index(inplace = True, drop = True)
product_new.drop(columns = ['NetProfit 2023', 'Gross Revenue 2023', 'RefundCost 2023', 'Units Sold 2023','Net Margin 2023' ], inplace = True)
df_All['Score'] = df_All[['Sales more than 100 units', 'Net Profit above $200',
       'Net Margin above 10%']].sum(axis = 1)

C:\Users\K-M\AppData\Local\Temp\ipykernel_7104\2547991594.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_new['Yearly Gross Revenue growth'] = product_new['Gross Revenue 2024'] > product_new['Gross Revenue 2023']
C:\Users\K-M\AppData\Local\Temp\ipykernel_7104\2547991594.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_new['Yearly NetProfit growth'] = product_new['NetProfit 2024'] > product_new['NetProfit 2023']
C:\Users\K-M\AppData\Local\Temp\ipykernel_7104\2547991594.py:4: SettingWit

In [33]:
for i in range(0, product_new.shape[0]):
    if product_new.loc[i, 'Score'] == 0 or product_new.loc[i, 'Score'] == 1:
        product_new.loc[i, 'Class'] = 'D' 
    if product_new.loc[i, 'Score'] == 2 or product_new.loc[i, 'Score'] == 3:
        product_new.loc[i, 'Class'] = 'C' 
    if product_new.loc[i, 'Score'] == 4:
        product_new.loc[i, 'Class'] = 'B' 
    if product_new.loc[i, 'Score'] == 5:
        product_new.loc[i, 'Class'] = 'A' 


for i in range(0, df_All.shape[0]):
    if df_All.loc[i, 'Score'] == 0:
        df_All.loc[i, 'Class'] = 'D' 
    if df_All.loc[i, 'Score'] == 1:
        df_All.loc[i, 'Class'] = 'C' 
    if df_All.loc[i, 'Score'] == 2:
        df_All.loc[i, 'Class'] = 'B' 
    if df_All.loc[i, 'Score'] == 3:
        df_All.loc[i, 'Class'] = 'A' 


df_All = df_A2[['Product Name', 'Date', 'Gross Revenue 2024',
       'NetProfit 2024', 'Units Sold 2024',
       'Net Margin 2024', 'Sales more than 100 units', 'Net Profit above $200',
       'Net Margin above 10%', 'Score', 'Class']]

In [34]:
product_new.sort_values(by = ['Score',  'Class', 'Gross Revenue 2024'], ascending = False).to_excel('Discontinuation Report.xlsx',
                                                                                                   sheet_name = 'Old Products')
with pd.ExcelWriter('Discontinuation Report.xlsx', engine='openpyxl', mode='a') as writer:
        df_All.sort_values(by = ['Score',  'Class', 'Gross Revenue 2024'], ascending = False).to_excel(writer, sheet_name= 'New products',  index = False)